# Azure AI Agentek Model Context Protocol (MCP) támogatással

Ez a jegyzetfüzet bemutatja, hogyan lehet Azure AI Agenteket használni Model Context Protocol (MCP) eszközökkel, hogy egy intelligens ügynököt hozzunk létre, amely külső MCP szervereket használhat a kibővített képességek érdekében.


## Szükséges NuGet csomagok telepítése

Először telepítenünk kell az Azure AI Agents Persistent csomagot, amely az Azure AI Agents használatához szükséges alapvető funkciókat biztosítja.


## Kulcs nélküli hitelesítés előnyei

Ez a jegyzetfüzet bemutatja a **kulcs nélküli hitelesítést**, amely számos előnnyel jár:
- ✅ **Nincs kezelendő API-kulcs** - Azure-alapú identitáshitelesítést használ
- ✅ **Fokozott biztonság** - Nincsenek titkok a kódban vagy a konfigurációban tárolva
- ✅ **Automatikus hitelesítő adatok forgatása** - Az Azure kezeli a hitelesítő adatok életciklusát
- ✅ **Szerepkör-alapú hozzáférés** - Azure RBAC-t használ a finomhangolt jogosultságokhoz

A `DefaultAzureCredential` automatikusan a legjobb elérhető hitelesítési forrást fogja használni:
1. Kezelt identitás (amikor az Azure-ban fut)
2. Azure CLI hitelesítő adatok (fejlesztés közben)
3. Visual Studio hitelesítő adatok
4. Környezeti változók (ha konfigurálva vannak)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Telepítse az Azure Identity csomagot az Azure szolgáltatások hitelesítéséhez a DefaultAzureCredential használatával.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Szükséges névtér importálása

Importáld a szükséges névtereket az Azure AI Agents és Azure Identity használatához.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI Agent kliens konfigurálása (Kulcs nélküli hitelesítés)

Állítsd be a konfigurációs változókat, és hozd létre a PersistentAgentsClient-et **kulcs nélküli hitelesítéssel**:
- **projectEndpoint**: Az Azure AI Foundry projekt végpontja
- **modelDeploymentName**: A telepített AI modell neve (GPT-4.1 nano)
- **mcpServerUrl**: Az MCP szerver URL-je (Microsoft Learn API)
- **mcpServerLabel**: Egy címke az MCP szerver azonosítására
- **DefaultAzureCredential**: Kezelt identitást, Azure CLI-t vagy más hitelesítési forrást használ (nincs szükség API kulcsokra)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## MCP Eszközdefiníció Létrehozása

Hozzon létre egy MCP eszközdefiníciót, amely csatlakozik a Microsoft Learn MCP szerverhez. Ez lehetővé teszi az ügynök számára, hogy hozzáférjen a Microsoft Learn tartalmakhoz és dokumentációhoz.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## AI ügynök létrehozása

Hozzon létre egy tartós AI ügynököt a megadott modell és MCP eszközök segítségével. Az ügynök a következőkkel van konfigurálva:
- A GPT-4.1 nano modell
- Utasítások az MCP eszközök használatához segítségnyújtás céljából
- Hozzáférés a Microsoft Learn MCP szerverhez


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Szál létrehozása és üzenet küldése

Hozz létre egy beszélgetési szálat, és küldj egy felhasználói üzenetet, amelyben az Azure OpenAI és az OpenAI közötti különbségről kérdezel. Ez tesztelni fogja az ügynök képességét az MCP eszközök használatára a pontos információk biztosítása érdekében.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## MCP eszköz erőforrásainak konfigurálása (kulcs nélküli)

Állítsa be az MCP eszköz erőforrásait. Egy valóban kulcs nélküli megközelítéshez eltávolíthatja az egyedi fejléceket, ha az MCP szerver támogatja az Azure identitás-alapú hitelesítést. Az alábbi példa bemutatja, hogyan lehet hozzáadni fejléceket, ha szükséges, de kulcs nélküli forgatókönyvek esetén ezek nem feltétlenül szükségesek.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Ügynök futtatásának indítása

Hozzon létre és indítson el egy futtatást a felhasználó üzenetének feldolgozására. Az ügynök a konfigurált MCP eszközöket és erőforrásokat fogja használni a válasz generálásához.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Futtatás figyelése és eszközjóváhagyások kezelése (Kulcs nélküli)

Figyeld az ügynök futási állapotát, és kezeld a szükséges eszközjóváhagyásokat. Ez a ciklus:
1. Várja, hogy a futtatás befejeződjön, vagy beavatkozásra legyen szükség
2. Automatikusan jóváhagyja az MCP eszközhívásokat, ha szükséges
3. Kulcs nélküli hitelesítés esetén a fejléc nem feltétlenül szükséges, ha az MCP szerver támogatja az Azure identitást


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Beszélgetési eredmények megjelenítése

Az összes üzenet lekérése és megjelenítése a szálban, beleértve a felhasználó kérdését és az ügynök válaszát is. Az üzenetek időbélyegekkel és szerepkör-jelölésekkel, időrendi sorrendben jelennek meg.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
